In [25]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [12]:
data = pd.read_csv('data_clean.csv')

y1 = pd.read_csv('y1_energie.csv')

y2 = pd.read_csv('y2_co2.csv')

In [28]:
X = data.values

X_tensor = torch.tensor(X, dtype=torch.float)

y1_tensor = torch.tensor(y1.values, dtype=torch.float)

y2_tensor = torch.tensor(y2.values, dtype=torch.float)

y_tensor = torch.cat([y1_tensor, y2_tensor], dim=1)

print('-'*30, '\nData array :\n', '-'*30, sep='')
print(X_tensor)
print('-'*30, '\nSiteEnergyUse(kBtu) :\n', '-'*30, sep='')
print(y1_tensor)
print('-'*30, '\nTotalGHGEmissions :\n', '-'*30, sep='')
print(y2_tensor)
print('-'*30, '\nConcat Tensor :\n', '-'*30, sep='')
print(y_tensor)


------------------------------
Data array :
------------------------------
tensor([[0.2513, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.2374, 0.0917, 0.1127,  ..., 0.0000, 0.0000, 0.0000],
        [0.1690, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0058, 0.0339, 0.0000,  ..., 1.0000, 0.0000, 0.0000],
        [0.0029, 0.0396, 0.0118,  ..., 0.0000, 0.0000, 0.0000],
        [0.0079, 0.0487, 0.0270,  ..., 0.0000, 0.0000, 0.0000]])
------------------------------
SiteEnergyUse(kBtu) :
------------------------------
tensor([[7226362.5000],
        [8387933.0000],
        [6794584.0000],
        ...,
        [5765898.0000],
        [ 719471.1875],
        [1152896.2500]])
------------------------------
TotalGHGEmissions :
------------------------------
tensor([[249.9800],
        [295.8600],
        [286.4300],
        ...,
        [223.5400],
        [ 22.1100],
        [ 41.2700]])
------------------------------
Concat Tensor :
----------------------

In [9]:
input_size = X_tensor.shape[1]

class BuildingEnergyModel(nn.Module):
    def __init__(self, input_size, output_size=2):
        super(BuildingEnergyModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = BuildingEnergyModel(input_size=input_size)

In [29]:
# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Example training loop
num_epochs = 100
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    optimizer.zero_grad()  # Reset gradients
    
    # Forward pass: feed your input tensor through the model
    outputs = model(X_tensor)
    
    # Calculate the loss between the model's predictions and the combined target tensor
    loss = criterion(outputs, y_tensor)
    
    # Backward pass: compute gradients
    loss.backward()
    
    # Update model parameters
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

Epoch [10/100], Loss: 8443338948608.0000
Epoch [20/100], Loss: 8443206828032.0000
Epoch [30/100], Loss: 8443054784512.0000
Epoch [40/100], Loss: 8442876526592.0000
Epoch [50/100], Loss: 8442667859968.0000
Epoch [60/100], Loss: 8442423541760.0000
Epoch [70/100], Loss: 8442137280512.0000
Epoch [80/100], Loss: 8441805930496.0000
Epoch [90/100], Loss: 8441424248832.0000
Epoch [100/100], Loss: 8440984371200.0000
